## Clustering to clean H1B Job Titles

In my analytics class, we learned how to use Google's OpenRefine to clean H1B Visa applications and their job title fields. We used the clustering functionality, which was super cool and made me wonder how the heck Google did that stuff. This project is an attempt to replicate the functionality to some degree. 

First, let's import the necessary packages.

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sklearn.cluster
import distance

The data is taken from governmental records of applications for HB1 visas. You can find it here: http://www.flcdatacenter.com/caseh1b.aspx. I saved it with a csv extension in Sublime.

First, let's load the data into a dataframe so we can steal the column we want.

In [4]:
df = pd.read_csv("H1B.csv")
df.dtypes

SUBMITTED_DATE           object
CASE_NO                  object
NAME                     object
ADDRESS                  object
ADDRESS2                 object
CITY                     object
STATE                    object
POSTAL_CODE              object
NBR_IMMIGRANTS            int64
BEGIN_DATE               object
END_DATE                 object
JOB_TITLE                object
DOL_DECISION_DATE        object
CERTIFIED_BEGIN_DATE     object
CERTIFIED_END_DATE       object
JOB_CODE                  int64
APPROVAL_STATUS          object
WAGE_RATE_1             float64
RATE_PER_1               object
MAX_RATE_1              float64
PART_TIME_1              object
CITY_1                   object
STATE_1                  object
PREVAILING_WAGE_1       float64
WAGE_SOURCE_1            object
YR_SOURCE_PUB_1         float64
OTHER_WAGE_SOURCE_1      object
WAGE_RATE_2             float64
RATE_PER_2               object
MAX_RATE_2              float64
PART_TIME_2              object
CITY_2  

We're going to be working with job titles. First, let's take a look at what our job titles look like so we can understand the problem. The distance formula we're going to use is a bit computationally expensive, we'll only use the first 300 jobs.

In [17]:
titles = pd.Series(df['JOB_TITLE'].unique())[:300]

In [18]:
titles.head(20)

0                  MOLECULAR BIOLOGIST
1          Principal Software Engineer
2       Associate Sales & Distribution
3                   resident physician
4              Market Research Analyst
5                        fellow doctor
6                      Systems Analyst
7                             Lecturer
8                       Sales Engineer
9                     Business Analyst
10             Radiologic Technologist
11                 Account Executive I
12    Imaging Business Support Manager
13                   Software Engineer
14                 Assistant Professor
15          Senior Computer Programmer
16                 Industrial Engineer
17                   MARINE TECHNICIAN
18                  Programmer Analyst
19             Postgraduate Researcher
dtype: object

As you can see if you scroll through the list, even once we've taken the unique titles out of the dataframe, there are overlapping positions. There are lower case and upper case, words switched around, misspellings, etc. If we want to make this data useful and visualize it, we'll need to clean this up.

First, we can change all of the terms to lowercase. We can argue that it's a good idea to keep the punctuation, but we'll remove it to make it easier on the clustering later on.

In [19]:
#Convert all titles to lower case
titles = titles.apply(lambda x: x.lower())

Ok, now our data is ready for clustering.

One way to calculate the similarity between strings is called the "Levenshtein Distance." Code borrowed from http://stats.stackexchange.com/questions/123060/clustering-a-long-list-of-strings-words-into-similarity-groups. More info on the distance formula here: https://rosettacode.org/wiki/Levenshtein_distance#Python. This may take a few minutes to run. 

In [20]:
lev_similarity = -1 * np.array([[distance.levenshtein(j1,j2) for j1 in titles] for j2 in titles])

So we've created a matrix (in array form) of the Levenshtein Distance of each job title from the other job titles in the original jobs array. Here's what it looks like:

In [21]:
lev_similarity

array([[  0, -22, -24, ..., -17, -17, -14],
       [-22,   0, -25, ..., -22, -14, -23],
       [-24, -25,   0, ..., -25, -23, -25],
       ..., 
       [-17, -22, -25, ...,   0, -15, -15],
       [-17, -14, -23, ..., -15,   0, -14],
       [-14, -23, -25, ..., -15, -14,   0]])

Now we'll cluster these values. Affinity Propagation seems to be the right algorithm for the job, since we've already calculated the Levenshtein Distances for our jobs array. The algorithm was first proposed for this purpose here:http://science.sciencemag.org/content/315/5814/972.

Affinity Propagation seems similar to K-Means, but instead of clustering and then re-iterating, the algorithm sends messages from data to other data to figure out what's close and what's not. K-Means, on the other hand, chooses random centroids (not the case in AP) and then figures out which points are closest. Info from here: http://www.psi.toronto.edu/affinitypropagation/faq.html.

Another super important (and helpful) feature of Affinity Propagation is that we don't need to specify the number of centroids / exemplars, which is key for the nature of our data set.

In [22]:
affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping = 0.5)
affprop.fit(lev_similarity)

AffinityPropagation(affinity='precomputed', convergence_iter=15, copy=True,
          damping=0.5, max_iter=200, preference=None, verbose=False)

Now that we fit the model, let's print out all of the clusters into a Pandas series (so we can index by a string).

In [36]:
def orderClusters(array):
    
    clusters = pd.Series()
    
    for cluster_id in np.unique(affprop.labels_):

        exemplar = array[affprop.cluster_centers_indices_[cluster_id]]

        cluster = np.unique(array[affprop.labels_==cluster_id])

        if exemplar not in clusters:
            clusters[exemplar] = cluster
            
    return clusters

In [37]:
clusters = orderClusters(titles)

counter = 0
for key in clusters.keys():
    print key , ': ' , clusters[key]
    counter += 1
    if counter >= 10:
        break

software engineer :  ['computer software engineer' 'housekeeper general'
 'principal software engineer' 'software developer'
 'software development engineer' 'software engineer'
 'software engineer (associate)' 'software engineer vii'
 'software engineering manager' 'staff software engineer'
 'study abroad advisor']
assistant professor :  ['adjunct/visiting professor' 'assistant coordinator' 'assistant professor'
 'assistant professor of mathematics' 'assistant professor of philosophy'
 'foreign student advisor' 'res assistant plastic surgery'
 'sales manager/data processing' 'visiting assistant professor'
 'visiting professor']
marine technician :  ['dental laboratory technician' 'engineering technician'
 'helpers electricians' 'marine technician' 'research technician'
 'resident physician' 'staff physician']
business development manager :  ['business area controller' 'business development manager'
 'imaging business support manager' 'product development scientist']
human resources di

Definitely not bad for a first run through! A lot of these make sense and probably caught some real errors.

If we adjust the "preference" argument, we can force the algorithm to employ more clusters. To see how changing the preference changes our cluster size, we'll run it a few different times to find the length of the dictionary (i.e. the number of exemplars).

In [39]:
affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping = 0.5, preference = -10)
affprop.fit(lev_similarity)
clusters = orderClusters(titles)
len(clusters)

154

In [40]:
affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping = 0.5, preference = -3)
affprop.fit(lev_similarity)
clusters = orderClusters(titles)
len(clusters)

263

The total number of data points that we grabbed is close to being reached, so we can see that as the preference approaches 0, we approach no clusters.

Now let's print out all of the clusters that have more than one member.

In [41]:
clusters = orderClusters(titles)

for key in clusters.keys():
    if len(clusters[key]) >= 2:
        print key , ': ' , clusters[key]

research assoicate  pediatrics :  ['research associate pediatrics' 'research assoicate  pediatrics']
programmer analyst :  ['programmer analyst' 'programmer/analyst']
postdoctoral research associate :  ['post doctoral research associate' 'post-doctoral research associate'
 'postdoctoral research associate' 'postdoctoral research assoicate']
research assistant :  ['research assistant' 'research assistant i' 'research associate']
senior analyst :  ['junior analyst' 'senior analyst']
computer programmer :  ['computer programer' 'computer programmer']
chemical engineer :  ['chemical engineer' 'electrical engineer' 'mechanical engineer']
technical specialist :  ['financial specialist' 'technical specialist']
post doctoral researcher :  ['post doctoral researcher' 'postdoctoral researcher']
architect :  ['architect' 'architect ']
manager computer operations :  ['manager compouter operations' 'manager computer operations']
associate director,  financial :  ['associate director,  financial' 'a

So how the heck are we supposed to know how many clusters are correct = what preference to use? Well, that's a great question, and the subject of this exact research paper from Cornell: https://arxiv.org/abs/0805.1096. The basic idea is – keep iterating until you converge on the right amount of clusters. It's called "Adaptive Affinity Propagation."

### Conclusions

1) To improve the accuracy of this project, we'd need to implement and adaptive framework to find the right preference value (= the optimal number of clusters).

2) For full effectiveness, we'd need to run the distance formula and clustering algorithm on every group, which would require some distributed computing on a cluster.